In [148]:
# import external libraries
%matplotlib inline
import collections
import inspect
import pickle
import re

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [136]:
# import local dependencies
from scripts.helpers import *
from scripts.plots import *
from scripts.data_manipulation import *
from scripts.data_analysis import *

***
**Exploratory data analysis**
***

Let's start by looking at Swiss *food* imports on a macro scale:

In [44]:
# Show data manipulation function helpers
%psource load_imports

def load_imports(filepath, sheet=0):
    """Loads specific imports/exports data from Swiss Impex"""
    column_names = ["commercial_partner", "quantity_kg", "value_chf", "value_change_%"]
    
    return pd.read_excel(
        filepath,
        sheet_name=sheet,
        names=column_names,
        usecols=[1, 2, 3, 4],
        skiprows=5,
        converters={
            # strip leading whitespace from country name
            0: lambda x: x.strip()
        }
    ).dropna(how='all')  # drop empty rows


In [45]:
%psource load_imported_food

def load_imported_food():
    """Load imports: food, beverages, and tobacco"""
    return load_imports(FOOD_IMPORTS_PATH)


In [160]:
imported_food = load_imported_food()
imported_food.head()

,quantity_kg,value_chf,value_change_%
commercial_partner,,,
Total trade,4.125947e+09,1.058994e+10,4.5
Afghanistan,3.300000e+01,1.229620e+05,244.1
Albania,3.444010e+05,3.499380e+05,-26.7
Algeria,9.049800e+04,4.043350e+05,0.1
Amer. Virgin,5.000000e+01,2.793000e+03,-90.6


Let's look at how much of the food, beverages, and tobacco that Switzerland imports comes from its nearest neighboring countries. The farther a country is, the more carbon emissions it would cost to import that food.

In [41]:
nearby_countries

['Portugal',
 'Spain',
 'France',
 'Germany',
 'Italy',
 'Austria',
 'Belgium',
 'Netherlands',
 'Czech Republic',
 'Slovenia',
 'Croatia']

In [40]:
%psource compute_nearby_imports_ratio

def compute_nearby_imports_ratio(imports):
    """Computes the ratio of imported products which come from nearby countries"""
    imported_nearby = imports[imports.commercial_partner.isin(nearby_countries)]
    total_nearby = imported_nearby.quantity_kg.sum()
    total_overall = imports.iloc[0, 1]
    return total_nearby / total_overall


In [9]:
ratio_nearby_food = compute_nearby_imports_ratio(imported_food)
print("{0:.1%} of imported food comes from nearby countries".format(ratio_nearby_food))

74.2% of imported food comes from nearby countries


Approximately 74% of Switzerland's imported food, beverage, and tobacco products come from fairly nearby countries. That's pretty good! It means imports have a lower carbon output than if Switzerland imported a lot from far away. It does not, however, indicate anything to us about whether the countries that Switzerland sources its imports from are carbon efficient. For example, Germany might have higher carbon intensities per kilo of wheat produced than the same product in France.

What about Switzerland's imported *feedstuffs* for animals? If the Swiss import a lot of feed, that contributes to an even higher carbon intensity for raising meat.

In [46]:
%psource load_imported_feed

def load_imported_feed():
    """Load imports: feeding stuffs for animals"""
    return load_imports(FOOD_IMPORTS_PATH, sheet=1)


In [10]:
imported_feed = load_imported_feed()
imported_feed.head()

,commercial_partner,quantity_kg,value_chf,value_change_%
0,Total trade,1.805791e+09,790556375.0,5
1,Albania,7.400000e+01,857.0,26.6
2,Argentina,1.153685e+06,396099.0,268.2
3,Australia,5.040000e+02,1627.0,-88.5
4,Austria,1.028771e+08,40044159.0,17.3


In [11]:
ratio_nearby_feed = compute_nearby_imports_ratio(imported_feed)
print("{0:.1%} of imported feed comes from nearby countries".format(ratio_nearby_feed))

74.0% of imported feed comes from nearby countries


Nearly three-quarters of imported animal feed is imported from nearby countries, which means when Switzerland must import feed, they mostly minimize carbon emissions from the process by reducing the travel distance of the feed. More importantly, a report from the State Secretariat for Economic Affairs ("Concentrate Animal Feed as an Input Good in Swiss Agricultural Production - The Effects of Border Protection and Other Support Measures") claims 90% of animal feed used in Switzerland is domestically produced, drastically reducing the carbon impact of the meat industry in the country. This percentage can be found in [this PDF document](https://www.sbv-usp.ch/fileadmin/sbvuspch/00_Bilder/06_Services/Agristat/Statistiken/Produktionsmittel__Umwelt/SES2018_Kap04_Produktionsmittel-Umwelt.pdf) (see the table on page 8):

We will need to pay attention to where food is coming from by animal type. For example, 90% of cow and cattle feed is domestically produced, whereas that number is only 52% for pork and 30% for poultry (found in the Concentrate Animal Feed report above, where it was sourced from the PDF linked above). So even if chickens are less carbon intensive, they require more imported feed which might increase their carbon output!

In [47]:
%psource load_imported_meat

def load_imported_meat():
    """Load imports: meat and edible meat offal"""
    return load_imports(MEAT_IMPORTS_PATH, sheet="02")


In [42]:
imported_meat = load_imported_meat()
imported_meat.head()

,commercial_partner,quantity_kg,value_chf,value_change_%
0,Total trade,77042136.0,688620381.0,-2
1,Albania,141.0,5676.0,*
2,Argentina,1460763.0,16806383.0,13.4
3,Australia,2801755.0,60160817.0,-10.1
4,Austria,6358938.0,44545918.0,-10.4


As we can see from the above table, Switzerland imported 77 million kg of meat and edible offal in 2018 (equivalent to 77 thousand tonnes).

Now let's look at what Switzerland's population _consumes_. 

In [92]:
%psource load_food_consumption

def load_food_consumption():
    colnames = [
        "food_type",
        "quantity_total_1k_tonnes",
        "quantity_kg_person_year",
        "protein_total_tonnes",
        "protein_g_person_day",
        "protein_%_local_production",
        "energy_intake_total_tj",
        "energy_intake_kj_person_day",
        "energy_intake_%_local_production"
    ]
    
    return pd.read_excel(
        CONSUMPTION_PATH,
        sheets=0,  # first sheet, year 2017
        names=colnames,
        index_col=0,
        skiprows=9,
        converters={
            # strip whitespace from first column (index)
            0: lambda x: x.strip()
        }
    ).dropna()


In [113]:
consumption = load_food_consumption()
consumption.head()

,quantity_total_1k_tonnes,quantity_kg_person_year,protein_total_tonnes,protein_g_person_day,protein_%_local_production,energy_intake_total_tj,energy_intake_kj_person_day,energy_intake_%_local_production
food_type,,,,,,,,
"Denrées alimentaires, total",7157.034169,841.014590,302451.392778,97.371792,70.553057,39794.586230,12811.546844,59.043218
Denrées alimentaires\nvégétales,4503.066844,529.150039,121985.710268,39.272318,41.448430,28153.732943,9063.867792,42.657761
Produits à base de céréales,767.003856,90.129713,71022.714316,22.865191,55.579067,8145.924472,2622.514841,58.419713
"Pommes de terre, etc.",364.760695,42.862596,5262.916468,1.694354,91.157487,853.361540,274.732882,88.965659
Amidons,29.015108,3.409530,128.987886,0.041527,0.000000,414.978520,133.598995,0.000000


Specifically, let's look at meat from the above table. How much meat did Swiss people consume in 2017?

In [126]:
total_meat_consumption = consumption.loc["Viande"].quantity_total_1k_tonnes
print(f"The Swiss population consumed {total_meat_consumption * 1000} tonnes of meat in 2017")

The Swiss population consumed 413332.55 tonnes of meat in 2017


As can be seen, the Swiss consumed 413 thousand tonnes, but they imported 77 thousand tonnes. This means most of the Swiss meat is produced within the country, or 81%.

Some other important facts, as provided by an article on swissinfo.ch: Switzerland produces more than enough milk and milk products to satisfy its needs, but has to import the majority of its fruits, vegetables, and nuts. Animal feed is one of the major imports, as well as staples like rice and wheat.

In [128]:
consumption.loc["Fruits"]

quantity_total_1k_tonnes             978.618988
quantity_kg_person_year              114.996351
protein_total_tonnes                4845.205647
protein_g_person_day                   1.559875
protein_%_local_production            12.266883
energy_intake_total_tj              1707.068693
energy_intake_kj_person_day          549.577031
energy_intake_%_local_production      18.527254
Name: Fruits, dtype: float64

In [134]:
%psource load_imported_fruits

def load_imported_fruits():
    """Load imports: fruits"""
    # bananas, exotic fruit, citrus fruit, grapes,
    # melons, apples, stone fruit, berries
    sheets = list(range(19, 27))
    return load_imports(FRUITS_VEGGIES_IMPORTS_PATH, sheet=sheets)


In [151]:
# imported_fruits = load_imported_fruits()

In [155]:
isinstance(imported_food, pd.DataFrame)

True